In [95]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as rx
import rioxarray as rix
import tensorflow as tf

2023-12-07 19:59:01.281203: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [85]:
# JUST FOR PLOTTING
def norm_diff(x,y):
    return (x-y)/(x+y)

child = "S2A_MSIL2A_20170701T093031_19_10"

def plot_folder(folder,rgb=True):
    parent = "data/"
    if rgb:
        plt.figure(figsize=(6, 6))
        blue = rix.open_rasterio(parent+f"{folder}/{folder}_B02.tif")
        green = rix.open_rasterio(parent+f"{folder}/{folder}_B03.tif")
        red = rix.open_rasterio(parent+f"{folder}/{folder}_B04.tif")
        print(blue.attrs)
        rgb_img = rx.concat([red, green, blue], dim='band')
        # rgb_img = (red+green+blue).squeeze()
        rgb_img.plot.imshow(robust=True)
        plt.title("RGB image")
        plt.show()

    elif not rgb:
        plt.figure(figsize=(6, 6))
        nir = rix.open_rasterio(parent+f"{folder}/{folder}_B8A.tif")
        swir = rix.open_rasterio(parent+f"{folder}/{folder}_B11.tif")
        avg_ndmi = np.mean(norm_diff(nir.values,swir.values).squeeze())
        ndmi_img = norm_diff(nir,swir).squeeze()
        print(avg_ndmi)
        # plt.imshow(ndmi)
        # plt.colorbar(label='Pixel Values')
        # plt.title('GeoTIFF Image')
        # plt.show()
        ndmi_img.plot.imshow(robust=True)
        plt.title(f"NDMI image. Average NDMI:{avg_ndmi:.2f}")
        plt.show()

# plot_folder(child,rgb=True)
# plot_folder(child,rgb=False)


In [76]:
import os

def list_folders(directory):
    folders = np.array([d for d in os.listdir(directory) if os.path.isdir(os.path.join(directory, d))])
    return folders
all_folders = list_folders("data/")
print(all_folders)
print(len(all_folders))


['S2A_MSIL2A_20170701T093031_19_10' 'S2B_MSIL2A_20170801T095029_4_23'
 'S2B_MSIL2A_20170906T101019_56_80' ... 'S2B_MSIL2A_20170801T095029_11_32'
 'S2B_MSIL2A_20170906T101020_78_48' 'S2B_MSIL2A_20170802T092029_10_37']
3324


In [77]:

# Read GeoTIFF file
def norm_diff(x,y):
    try:
        return (x-y)/(x+y)
    except Exception as e:
        print(e)
        return None

# testchild = "S2A_MSIL2A_20171101T094131_1_38"


def get_input_label(folder):
    try:
        base_path=f"data/{folder}/{folder}"

        blue = rix.open_rasterio(f"{base_path}_B02.tif")
        green = rix.open_rasterio(f"{base_path}_B03.tif")
        red = rix.open_rasterio(f"{base_path}_B04.tif")     

        rgb = np.concatenate([red.values,green.values,blue.values], axis=0)

        # print(rgb)
        nir = rix.open_rasterio(f"{base_path}_B8A.tif")
        swir = rix.open_rasterio(f"{base_path}_B11.tif")

        ndmi = norm_diff(nir.values.squeeze(),swir.values.squeeze())
        label = np.mean(ndmi)
        
        return rgb, label
    except Exception as e:
        print(e)
        return None, None

In [81]:
folders = all_folders.copy()

input = np.zeros((len(folders), 3, 120, 120))
labels = np.zeros((len(folders), 1))

for j, folder in enumerate(folders):
    rgb, label = get_input_label(folder)
    # print(label)
    input[j] = rgb
    labels[j] = label


/var/folders/68/x8klff196fl8_dk1n2t0qbtc0000gn/T/ipykernel_8267/3558035866.py:4: RuntimeWarning: invalid value encountered in divide
  return (x-y)/(x+y)
/var/folders/68/x8klff196fl8_dk1n2t0qbtc0000gn/T/ipykernel_8267/3558035866.py:4: RuntimeWarning: invalid value encountered in divide
  return (x-y)/(x+y)


data/S2B_MSIL2A_20170802T092029_18_24/S2B_MSIL2A_20170802T092029_18_24_B03.tif: No such file or directory


/var/folders/68/x8klff196fl8_dk1n2t0qbtc0000gn/T/ipykernel_8267/3558035866.py:4: RuntimeWarning: invalid value encountered in divide
  return (x-y)/(x+y)
/var/folders/68/x8klff196fl8_dk1n2t0qbtc0000gn/T/ipykernel_8267/3558035866.py:4: RuntimeWarning: invalid value encountered in divide
  return (x-y)/(x+y)


data/S2A_MSIL2A_20171121T112351_11_54/S2A_MSIL2A_20171121T112351_11_54_B04.tif: No such file or directory


/var/folders/68/x8klff196fl8_dk1n2t0qbtc0000gn/T/ipykernel_8267/3558035866.py:4: RuntimeWarning: invalid value encountered in divide
  return (x-y)/(x+y)
/var/folders/68/x8klff196fl8_dk1n2t0qbtc0000gn/T/ipykernel_8267/3558035866.py:4: RuntimeWarning: invalid value encountered in divide
  return (x-y)/(x+y)
/var/folders/68/x8klff196fl8_dk1n2t0qbtc0000gn/T/ipykernel_8267/3558035866.py:4: RuntimeWarning: invalid value encountered in divide
  return (x-y)/(x+y)


In [89]:
# remove rows from label and input where label is nan or invalid (>1 or <-1)
faulty_row = (np.isnan(labels) | (np.abs(labels)>1)).squeeze()
print(folders.shape)
labels_clean = labels[~faulty_row]
input_clean = input[~faulty_row]
folders_clean = folders[~faulty_row]
# labels = np.clip(labels, -1, 1)
print(len(labels))
print(len(labels_clean))
print(len(input_clean))
# print(input[:3])
# print(labels[:3])
# print(input.shape)
np.save("arrays/clean_folders.npy",folders_clean)
np.save("arrays/clean_labels.npy",labels_clean)
np.save("arrays/clean_input.npy",input_clean)

(3324,)
3324
1284
1284


In [98]:
from ResNet50 import DNN_model
# start constructing network for RGB dimension (120,120,3)
input_clean = input_clean.reshape(len(input_clean),120,120,3)
input_shape = input_clean.shape

# Example parameters, replace with your actual values
img_placeholder = tf.placeholder(tf.float32, input_shape)
num_classes = 10  # Replace with your actual number of classes
is_training = True  # Set to False for inference
prediction_threshold = 0.5  # Set the threshold based on your needs

# Instantiate the model
model = DNN_model(img_placeholder, num_classes, is_training, prediction_threshold)
model.create_network()

AttributeError: module 'tensorflow' has no attribute 'GraphKeys'

In [ ]:
with tf.Session() as sess:
    # Initialize variables
    sess.run(tf.global_variables_initializer())

    # Example input data, replace with your actual data
    input_data = input_clean

    # Run the model
    predictions = sess.run(model.predictions, feed_dict={model.img: input_data})
    print("Predictions:", predictions)